In [3]:
# !apt-get install p7zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [5]:
# !wget https://github.com/samsungnlp/smiler/raw/main/smiler_corpora_v1.0_part1.7z
# !7z e smiler_corpora_v1.0_part1.7z

--2024-05-02 05:11:16--  https://github.com/samsungnlp/smiler/raw/main/smiler_corpora_v1.0_part1.7z
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/samsungnlp/smiler/main/smiler_corpora_v1.0_part1.7z [following]
--2024-05-02 05:11:16--  https://raw.githubusercontent.com/samsungnlp/smiler/main/smiler_corpora_v1.0_part1.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28929993 (28M) [application/octet-stream]
Saving to: ‘smiler_corpora_v1.0_part1.7z’

smiler_corpora_v1.0 100%[===================>]  27.59M  11.5MB/s    in 2.4s    

2024-05-02 05:11:19 (11.5 MB/s) - ‘smiler_corpora_v1.0_part1.7z’ 

In [1]:
SMILER_TRANSLATE = {
    "en": {
        "birth-place": "birth place",
        "eats": "eats",
        "event-year": "event year",
        "first-product": "first product",
        "from-country": "from country",
        "has-author": "has author",
        "has-child": "has child",
        "has-edu": "has education",
        "has-genre": "has genre",
        "has-height": "has height",
        "has-highest-mountain": "has highest mountain",
        "has-length": "has length",
        "has-lifespan": "has lifespan",
        "has-nationality": "has nationality",
        "has-occupation": "has occupation",
        "has-parent": "has parent",
        "has-population": "has population",
        "has-sibling": "has sibling",
        "has-spouse": "has spouse",
        "has-tourist-attraction": "has tourist attraction",
        "has-type": "has type",
        "has-weight": "has weight",
        "headquarters": "headquarters",
        "invented-by": "invented by",
        "invented-when": "invented when",
        "is-member-of": "is member of",
        "is-where": "located in",
        "loc-leader": "location leader",
        "movie-has-director": "movie has director",
        "no_relation": "no relation",
        "org-has-founder": "organization has founder",
        "org-has-member": "organization has member",
        "org-leader": "organization leader",
        "post-code": "post code",
        "starring": "starring",
        "won-award": "won award",
    },
    "fr": {
        "birth-place": "lieu de naissance",
        "event-year": "année de l'événement",
        "from-country": "du pays",
        "has-author": "a un auteur",
        "has-child": "a un enfant",
        "has-edu": "a une éducation",
        "has-genre": "a un genre",
        "has-occupation": "a une profession",
        "has-parent": "a un parent",
        "has-population": "a de la population",
        "has-spouse": "a un conjoint",
        "has-type": "a le type",
        "headquarters": "siège social",
        "is-member-of": "est membre de",
        "is-where": "situé à",
        "loc-leader": "guide d'emplacement",
        "movie-has-director": "le film a un réalisateur",
        "no_relation": "aucune relation",
        "org-has-founder": "l'organisation a un fondateur",
        "org-has-member": "l'organisation a un membre",
        "org-leader": "chef d'organisation",
        "won-award": "a remporté le prix",
    },
    "ru": {
        "event-year": "год события",
        "has-edu": "имеет образование",
        "has-genre": "имеет жанр",
        "has-occupation": "имеет профессию",
        "has-population": "имеет население",
        "has-type": "имеет тип",
        "is-member-of": "является членом",
        "no_relation": "без связи",
    }
}

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM 
class args:
    tgt_num = 200 # First 200 examples from the target language
    src_num = -1 # NO examples from the source language
    max_new_tokens = 100
    temperature = 0.6
    device="cuda:0"
    l1 = "en"
    l2 = "fr"
    l3 = "ru"

tokenizer = AutoTokenizer.from_pretrained("bigscience/mt0-large")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large").to(args.device)

In [13]:
## Script for creating a prompt for the user to enter a number
import json
import argparse
import csv
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
# XGLMTokenizer, XGLMForCausalLM,
from tqdm import tqdm
import re


dir_path=""
prompt_template_zero = {
    "en": """
Consider the following statement as truth: {}.
What is the most appropriate relation between {} and {} from the following options?

{}

Please choose the most suitable option from the given choices.
    """,
# "en":"""
# {} Which of the following relations best describes the connection between {} and {}?
# Options:

# {}

# Provide your answer as the option mentioned above
# """,
    "fr":"""
Considérez l'énoncé suivant comme vérité : {}.
Quelle est la relation la plus appropriée entre {} et {} parmi les options suivantes ?

{}

Veuillez choisir l'option la plus appropriée parmi les choix donnés
    """,
    "ru":"""
Рассматривайте следующее утверждение как истинное: {}.
Какое из следующих отношений наиболее подходит для описания связи между {} и {}?

{}

Пожалуйста, выберите наиболее подходящий вариант из предложенных
    """
}

prompt_template_example = """
Consider the following statement as truth: {}'
What is the most appropriate relation between {} and {} from the following options?

{}

Please choose the most suitable option from the given choices.

Examples:
{}

"""



def create_examples(src, tgt, num, choices):
    if num == -1:
        return prompt_template_zero[tgt]

    with open(dir_path + f"{lang}_corpora_train.tsv", "r") as f:
        reader = csv.reader(f, delimiter="\t")
        examples = ""
        for i, row in enumerate(reader):
            if i < 1:
                continue
            if i > num:
                break
            sen = row[4]
            sen = sen.replace("<e1>", "").replace("</e1>", "").replace("<e2>", "").replace("</e2>", "")
            e1 = row[4].split("<e1>")[1].split("</e1>")[0]
            e2 = row[4].split("<e2>")[1].split("</e2>")[0]
            sen = "{{"+sen+"}}"

            examples += "For statement {}: {} is related to {} as: {}\n".format(sen, e1, e2, row[3])

        prompt_template_example.format("{}", "{}", "{}", "{}", examples)
        return examples



def inference(prompt, max_new_tokens=args.max_new_tokens, temperature=args.temperature):


    model_inputs = tokenizer([prompt], return_tensors="pt").to(args.device)

    # For removal of input string from the output string
    input_ids_cutoff = model_inputs.input_ids.size(dim=1)

    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens=max_new_tokens,
                                   top_p = 1,
                                   temperature = temperature,
                                  #  num_beams=20,
                                   do_sample=True,
                                   pad_token_id=tokenizer.eos_token_id)

    completion = tokenizer.decode(
    generated_ids[0],
    skip_special_tokens=True)

    return completion


def create_rel_vocab(src_lang, tgt_lang, src_num, tgt_num):
    relations = set()

    # Iterate src
    with open(dir_path + f"{src_lang}_corpora_train.tsv", "r") as f:
        reader = csv.reader(f, delimiter="\t")
        for i, row in enumerate(reader):
            if i < 1:
                continue
            if i > src_num:
                break
            relations.add(f"{row[3]}")

    # Iterate tgt
    with open(dir_path + f"{tgt_lang}_corpora_test.tsv", "r") as f:
        reader = csv.reader(f, delimiter="\t")
        for i, row in enumerate(reader):
            if i < 1:
                continue
            if i > tgt_num:
                break
            rel = "{{"+SMILER_TRANSLATE[tgt_lang][row[3]]+"}}"
            relations.add(f'{rel}')
            # relations.add(f'"{row[3]}"')

    return relations


def main():

    for lang1 in [args.l2]:
        for lang2 in [args.l3]:
            actual_result = []
            gen_result = []

            if lang1 == lang2:
                continue

            # Create relation vocabulary
            choices = create_rel_vocab(lang1, lang2, args.src_num, args.tgt_num)

            # If not zero-shot, create examples
            prompt_src = create_examples(lang1, lang2, args.src_num, choices)

            os.makedirs(f"{lang1}_{lang2}", exist_ok=True)
            # Create prompt for each example and save it
            with open(dir_path + f"{lang2}_corpora_test.tsv", "r") as f:
                reader = csv.reader(f, delimiter="\t")
                for i, row in enumerate(reader):
                    if i < 1:
                        continue
                    if i > args.tgt_num:
                        break
                    # if f'"{SMILER_TRANSLATE[lang2][row[3]]}"' not in choices:
                    # # if f'"{row[3]}"' not in choices:
                    #     raise ValueError(f"Relation {row[3]} not in the relation vocabulary {lang2}")

                    sen = row[4]
                    sen = sen.replace("<e1>", "").replace("</e1>", "").replace("<e2>", "").replace("</e2>", "")
                    e1 = row[4].split("<e1>")[1].split("</e1>")[0]
                    e2 = row[4].split("<e2>")[1].split("</e2>")[0]
                    sen = "{{"+sen+"}}"
                    e1 = "{{"+e1+"}}"
                    e2 = "{{"+e2+"}}"
                    prompt = prompt_src.format(sen, e1, e2, "\n".join(choices))
                    result = inference(prompt)
                    # Clean the result
                    result = re.sub(r"[!@#$%^&*()_+\-=[\]{};':,.<>/?\\|~]", "", result)
                    # Also remove all quotes
                    result = result.replace('"', "").replace("'", "").replace("`", "")
                    # result = next((key for key, value in SMILER_TRANSLATE[lang2].items() if value == result), result)

                    print(row[3], result)
                    
                    with open(f"{lang1}_{lang2}/prompt_{i}.txt", "w") as f:
                        f.write(prompt)

                    actual_result.append(SMILER_TRANSLATE[lang2][row[3]])
                    gen_result.append(result)

            df = pd.DataFrame({"actual": actual_result, "generated": gen_result})
            df.to_csv(f"{lang1}_{lang2}/results.tsv", sep="\t", index=False)
            print(f"Results saved for {lang1} to {lang2}")


main()

has-occupation अभिनेत्री
has-occupation  birth date is 14 september 1973  occupation is singer musician producer actor entrepreneur  birth place is Brooklyn New York United States
has-occupation director
has-population No
has-occupation  known for is a British musician He is a British musician He is a British composer and songwriter He is best known for his work in the theatre theatre and theatre
has-population Yes
has-occupation  yearsactive is 2005  occupation is actor  birth place is hollywood california
has-occupation  occupation is a religious critic
has-type  known for is knight  birth date is 13 juin 1928  nationality is italy  death date is 11 July 2015  birth place is bologna italy  name is carlo fascista
has-edu                                                  
has-population Yes
has-population а а
has-occupation  fields  电影 cinema cinema film cinema cinema  birth date  occupation  death date  birth place  name
has-type năm
has-occupation  name
has-occupation  yearsactive is 